In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1717480/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_15_1,0.999821,0.831591,0.807215,0.999410,0.000775,1.126150,0.689582,0.003877,0.020238,0.027848,1.011463,0.028286,144.324039,285.659212,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_15_0,0.999821,0.831591,0.807215,0.999410,0.000775,1.126150,0.689582,0.003877,0.020238,0.027848,1.011463,0.028286,144.324026,285.659199,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_45_0,0.999821,0.831591,0.807215,0.999853,0.000775,1.126150,0.689582,0.000470,0.020238,0.027848,1.011463,0.028286,144.324014,285.659187,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_19_6,0.999821,0.831591,0.807215,0.999703,0.000775,1.126150,0.689582,0.001295,0.020238,0.027848,1.011463,0.028286,144.324014,285.659187,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_19_5,0.999821,0.831591,0.807215,0.999703,0.000775,1.126150,0.689582,0.001295,0.020238,0.027848,1.011463,0.028286,144.324014,285.659187,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,model_1_1_2,0.930921,0.757859,0.752547,0.954970,0.299097,1.619195,0.885124,0.333638,0.836414,0.546898,5.421087,0.555511,132.413975,273.749148,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
488,model_1_1_1,0.924509,0.753667,0.729323,0.946726,0.326859,1.647227,0.968196,0.394723,0.953746,0.571716,5.831450,0.580720,132.236453,273.571626,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
489,model_1_1_0,0.923635,0.752610,0.728768,0.945963,0.330640,1.654300,0.970180,0.400377,0.954492,0.575013,5.887342,0.584069,132.213450,273.548622,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
490,model_1_0_1,0.913303,0.747994,0.717616,0.854313,0.375377,1.685167,1.010072,0.383833,1.123052,0.612680,6.548624,0.622330,131.959647,273.294819,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
